# This notebook does scrape the investment data of companies that are in the space sectors.

### It takes :
#### the date of investment
#### The amount invested (and translate it ito dollars)
#### The investors involved in the fundrising
#### The type of fundrising (seed, A, B, C, IPO, ...)

In [1]:
import pandas as pd
import numpy as np
import random
from random import randint
from difflib import SequenceMatcher
import re
import datetime
from difflib import SequenceMatcher

## Used for google and linkedin part
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
print('libraries loaded')

## Used for scraping
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
print('Scrapping libraries imported')

print('Libraries imported')

libraries loaded
Scrapping libraries imported
Libraries imported


In [2]:
df = pd.read_excel('C:/passto/dataset.xlsx',
                encoding='ISO-8859-1')

In [3]:
companies = df['Organization Name'].tolist()
companies_test = companies[0:3]
companies_test

['Cape analytics', 'DISH network', 'Swift Navigation']

# Aggregating location
Returns the country of the company as we are not interested at the city or region (aggregate for EU)

In [4]:
location = df['Headquarters Location'].tolist()
country_clean = []
for item in location:
    if type(item) != 'str':
        item=str(item)
        country_clean.append(item)
    else:
        country_clean.append(item)
country_agg = []
for item in country_clean:
    if 'China' in item:
        country_agg.append('China')
    elif 'Russian Federation' in item:
        country_agg.append('Russia')
    elif 'United States' in item:
        country_agg.append('USA')
    elif 'India' in item:
        country_agg.append('India')
    elif 'Japan' in item:
        country_agg.append('Japan')
    elif 'France' in item:
        country_agg.append('EU')
    elif 'United Kingdom' in item:
        country_agg.append('EU')
    elif 'Italy' in item:
        country_agg.append('EU')
    elif 'Spain' in item:
        country_agg.append('EU')
    elif 'Spain' in item:
        country_agg.append('EU')
    elif 'Luxembourg' in item:
        country_agg.append('EU')
    elif 'Switzerland' in item:
        country_agg.append('EU')
    elif 'Poland' in item:
        country_agg.append('EU')
    elif 'Germany' in item:
        country_agg.append('EU')
    elif 'Ireland' in item:
        country_agg.append('EU')
    elif 'Finland' in item:
        country_agg.append('EU')
    elif 'Denmark' in item:
        country_agg.append('EU')
    elif 'Sweden' in item:
        country_agg.append('EU')
    elif 'Norway' in item:
        country_agg.append('EU')
    elif 'Czech Republic' in item:
        country_agg.append('EU')
    elif 'Estonia' in item:
        country_agg.append('EU')
    elif 'Latvia' in item:
        country_agg.append('EU')
    elif 'Netherlands' in item:
        country_agg.append('EU')
    elif 'Belgium' in item:
        country_agg.append('EU')
    elif 'Bulgaria' in item:
        country_agg.append('EU')
    elif 'Croatia' in item:
        country_agg.append('EU')
    elif 'Hungary' in item:
        country_agg.append('EU')
    elif 'Greece' in item:
        country_agg.append('EU')
    elif 'Slovenia' in item:
        country_agg.append('EU')
    elif 'Portugal' in item:
        country_agg.append('EU')
    elif 'Austria' in item:
        country_agg.append('EU')
    elif 'Slovakia' in item:
        country_agg.append('EU')
    elif 'Lithuania' in item:
        country_agg.append('EU')
    elif item=='nan':
        country_agg.append('Unknown')
    else:
        country_agg.append('ROW')

In [5]:
country_agg_test = country_agg[0:3]
country_agg_test

['USA', 'USA', 'USA']

In [7]:
Label = df['Label'].tolist()
Label_test = Label[0:3]

In [6]:
# Add the country (aggregated by world great regions) to the initial dataset
df['country'] = country_agg

In [361]:
# Saving to excel
df.to_excel('C:/yourway/df_compagnies.xlsx', index=None, header= True)

# Function used during scraping

In [10]:
def formating_dates(df):
    '''A function that allow to keep only the year of investment as we are not
    interested to the month/day of investment'''
    
    announced = df['Announced Date']
    Announced_date=[]
    for date in announced:
        date = date.split(', ')[1]
        Announced_date.append(date)
    df['Announced_date'] = Announced_date
    return df

In [11]:
def doublon_missing_val_handling(df):
    '''A function that encode string - signs as missing values and
    discard duplicate values'''
    
    df['Money Raised'] = df['Money Raised'].replace({'—': np.nan})
    df = df[~((df[['Announced_date']].duplicated(keep=False)) & (df.isnull().any(axis=1)))]
    df = df.replace(np.nan, '—', regex=True)
    return df

In [12]:
def conversion (currency,item):
    '''A function that translate the investment values into US dollars'''
    
    if currency =='$':
        item = item*1
    elif currency == "A$":
        item = item*0.78
    elif currency =='CA$':
        item = item*0.79
    elif currency == 'CN¥':
        item = item*0.15
    elif currency == '€':
        item = item*1.22
    elif currency == '£':
        item = item*  1.37
    elif currency == '₹':
        item = item*0.014
    elif currency == '¥':
        item = item*0.0097
    elif currency == 'NZ$':
        item = item*0.72
    elif currency == 'SGD':
        item = item*0.76
    elif currency == 'R$':
        item = item*0.18
    elif currency == 'CHF':
        item = item*1.1
    return item

In [13]:
def formating(item):
    '''A function that convert K, M, B to the right amount of 0 to make it suitable for passage to integers'''
    if 'M' in item:
        item = item.replace('M','000000')
        if '.' in item:
            item = item.replace('.','')
            item = item[:-1]
        item = int(item)
    elif 'K' in item:
        item = item.replace('K','000')
        if '.' in item:
            item = item.replace('.','')
            item = item[:-1]
        item = int(item)
    elif 'B' in item:
        item = item.replace('B', '000000000')
        if '.' in item:
            item = item.replace('.','')
            item = item[:-1]
        item = int(item)
    return item

In [14]:
def clean_up_dataset(df):
    '''A function that discard the currency sign and convert the amount from string to integer'''
    
    df['Announced Date'] = pd.to_datetime(df['Announced Date']).dt.strftime('%Y')
    money = df['Money Raised'].tolist()
    dates = df['Announced Date'].tolist()
   
    money_invested=[]
    for item in money:
        if '—' in item:
            item = item.replace('—', 'NaN')
        elif '$' in item:
            if item[0]=='A':
                item = item[2:]
                currency = 'A$'
                item = formating(item)
                item = conversion(currency, item)
            elif item[0]=='C':
                item = item[3:]
                currency = 'CA$'
                item = formating(item)
                item = conversion(currency, item)
            elif item[0] == 'N':
                item = item[3:]
                currency = 'NZ$'
                item = formating(item)
                item = conversion(currency, item)
            elif item[0] == '$':
                item = item[1:]
                currency = '$'
                item = formating(item)
                item = conversion(currency, item)
            elif item[0]=='R':
                item = item[2:]
                currency = 'R$'
                item = formating(item)
                item = conversion(currency, item)
        elif '¥' in item:
            if item[0]=='C':
                item = item[3:]
                currency = 'CN¥'
                item = formating(item)
                item = conversion(currency, item)
            else:
                item = item[1:]
                currency = '¥'
                item = formating(item)
                item = conversion(currency, item)
        else:
            if '€' in item:
                item = item[1:]
                currency = '€'
                item = formating(item)
                item = conversion(currency, item)
            elif '£' in item:
                item = item[1:]
                currency = '£'
                item = formating(item)
                item = conversion(currency, item)
            elif '₹' in item:
                item = item[1:]
                currency = '₹'
                item = formating(item)
                item = conversion(currency, item)
            elif 'S' in item:
                item = item[3:]
                currency = 'SGD'
                item = formating(item)
                item = conversion(currency, item)
            elif 'CHF' in item:
                item = item[3:]
                currency = 'CHF'
                item = formating(item)
                item = conversion(currency, item)
        money_invested.append(item)
    df['Money'] = money_invested
    return df

# Connecting to webdriver to start the scraping

In [18]:
driver = webdriver.Chrome('C:/yourway/chromedriver.exe')

In [19]:
driver.get('https://www.crunchbase.com/logout')

username = driver.find_element_by_name('email')
username.send_keys('your_username')
sleep(randint(5,7))

password = driver.find_element_by_name('password')
password.send_keys('your password')
sleep(randint(2,4))

In [20]:
########## Initiate emplty lists to receive the scraping outcome #########################################################
list_df_comp = [] ###### Will receive the data of investments made for each companies
list_df_investors = [] ######## Will receive the list of investors within each companies
list_df_investments_made = [] ##### Will receive the investment made by each companies
list_acquisitions = []######## Will receive the acquisition  of each companies 
i=0
while i<len(companies):
    
    ########################################################################
    ############## Get the search bar and type the name of the company #####
    search_bar = driver.find_element_by_id('mat-input-0')
    search_bar.clear()
    search_bar.send_keys(companies[i])
    sleep(randint(3,4))
    search_bar.send_keys(Keys.ENTER)
    sleep(randint(5,8))
    nb_result = driver.find_elements_by_class_name('component--results-info')
    #############################################################################
    #############################################################################
    
    ##############################################################################
    ########## Start Get into company's profile ##################################
    ##############################################################################
    
    
    ############## Check the number of results for the company name after ########
    
    if 'PREV' in nb_result[1].text:
        b = 40
        get_into_comps = []
        h=1
        while h<b:
            ############ Get the name of the companies in the research result #######################
            a=str(h)
            get_into_comp = driver.find_element_by_xpath('/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/discover/page-layout/div/div/div[2]/section[2]/results/div/div/div[3]/sheet-grid/div/div/grid-body/div/grid-row['+a+']'+'/grid-cell[2]/div/field-formatter/identifier-formatter/a')
            get_into_comps.append(get_into_comp)
            h=h+1
            ratios = []
        for get_into_comp in get_into_comps:
            ############### check which name is the closest of the one entered in the search bar #################
            ratios.append(SequenceMatcher(None, companies[i], get_into_comp.text).ratio())
        index_max = np.argmax(ratios)
        ################ Click on the link associated with the company whose name has the highest similarity ######
        get_into_comps[index_max].click()
    else: #### If there is only one page (less than 40 results but more than one: we repeat the same process)
        nb = nb_result[1].text.split(' res')[0]
        b = int(nb)
        if b>1:
            get_into_comps = []
            h=1
            while h<=b:
                a=str(h)
                get_into_comp = driver.find_element_by_xpath('/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/discover/page-layout/div/div/div[2]/section[2]/results/div/div/div[3]/sheet-grid/div/div/grid-body/div/grid-row['+a+']'+'/grid-cell[2]/div/field-formatter/identifier-formatter/a')
                get_into_comps.append(get_into_comp)
                h=h+1
            ratios = []
            for get_into_comp in get_into_comps:
                ratios.append(SequenceMatcher(None, companies[i], get_into_comp.text).ratio())
            index_max = np.argmax(ratios)
            get_into_comps[index_max].click()
        else: ################################# There is only one company ##############
            get_into_comp = driver.find_element_by_xpath('/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/discover/page-layout/div/div/div[2]/section[2]/results/div/div/div[3]/sheet-grid/div/div/grid-body/div/grid-row[1]/grid-cell[2]/div/field-formatter/identifier-formatter/a')
            get_into_comp.click()
            
            
            ##############################################################################
            ########## End Get into company's profile ####################################
            ##############################################################################
            
            
            ##############################################################################
            ########## Start searching if investment were made or not ####################
            ##############################################################################
    
    sleep(randint(2,4))
    get_status = driver.find_elements_by_xpath("//span[@class='component--field-formatter field-type-enum ng-star-inserted']")
    status = get_status[1].text.strip()
    clickables = driver.find_elements_by_class_name('ng-star-inserted')
    sleep(randint(2,4))
    ok_clickable_funding =[]
    for clickable in clickables:
        if clickable.text.strip()=='Total Funding Amount':
            ok_clickable_funding.append(clickable)
    if len(ok_clickable_funding)==0: ## Non link related to funding were found --> create an empty dataset ###
        df_comp = pd.DataFrame(columns=['Announced Date', 'Transaction Name', 'Number of Investors', 'Money Raised', 'Lead Investors'])
        df_comp = df_comp.append({'Announced Date': 'Dec 2, 2030', 'Transaction Name': 'none', 'Number of Investors': 0, 'Money Raised': '$0', 'Lead Investors': 'none'}, ignore_index=True)
        ok_clickable_investors = []
        for clickable in clickables:
            if clickable.text.strip()=='Number of Investors':
                ok_clickable_investors.append(clickable)
        if len(ok_clickable_investors) == 0: ## No link related to investors were found --> create an empty dataframe
            df_investors = pd.DataFrame(columns=['Investor Name', 'Lead Investor', 'Funding Round', 'Partners'])
            df_investors = df_investors.append({'Investor Name': 'none', 'Lead Investor': 'none', 'Funding Round': 'none', 'Partners': 'none'}, ignore_index=True)
        else : ### A link to investors were found --> get in
            ok_clickable_investors[0].click()
            sleep(randint(5,8))
            page_html = driver.page_source ## Charge the source code
            df_list = pd.read_html(page_html) ## Take a list of tables
            df_investors = df_list[1] ## Select the table associated with investors 
            
            ################################################################################
            ############# INVESTMENTS WERE MADE ############################################
            ################################################################################
    else:
        ok_clickable_funding[0].click() # click on the link associated to investments
        sleep(randint(5,8))
        page_html = driver.page_source
        checking_list_df = pd.read_html(page_html) 
        print(len(checking_list_df))
        #####################################################################################
        ## The len of the number of tables give information on which info are available, ##
        ## The first one is related to investment received, the second one to investors, ##
        ## the third to investment made, the forth to acquisition made ######################
        #####################################################################################
        
        if len(checking_list_df)>2: ## We might have data abount acquisition and investments made
            print(companies[i])
            titles = driver.find_elements_by_class_name('section-title')
            acquisitions = []
            for title in titles:
                if title.text.strip() =='Acquisitions':
                    acquisitions.append(title.text.strip())
            if len(acquisitions)>0: ## If acquisition is written somewhere on the page
                        df_acquisition = checking_list_df[-1] ## The df is the last table given by the function pd.read_html
                        buyer = [companies[i]]*len(df_acquisition)
                        df_acquisition['Buyer'] = buyer
                        list_acquisitions.append(df_acquisition)
                ######################
                        df_investments_made = checking_list_df[-2]
                        comp_name_investing = [companies[i]]*len(df_investments_made)
                        df_investments_made['Investing_company'] = comp_name_investing
                        list_df_investments_made.append(df_investments_made)
            else: ## If acquisition is not present on the page
                print('else was taken into account')
                df_investments_made = checking_list_df[2] # The investment made is the third table read by the function
                comp_name_investing = [companies[i]]*len(df_investments_made)
                df_investments_made['Investing_company'] = comp_name_investing
                list_df_investments_made.append(df_investments_made)
                
                ###################################################################################################
                ## Check the lenth of the table to know if we must click to unwrap them to charge the whole data ##
                ###################################################################################################
                
        clickables2 = driver.find_elements_by_class_name('mat-button-wrapper')
        ok_clickable2 =[]
        for clickable in clickables2:
            if clickable.text.strip()=='VIEW ALL':
                ok_clickable2.append(clickable)
        if len(ok_clickable2)==0: # All table are full length no need to unwrap
            df_list = pd.read_html(page_html)
            ###########################################################################
            ## Check if we have investment, and/or investors ##########################
            ##########################################################################
            if len(df_list)==1:
                df_comp = df_list[0]
                df_investors = pd.DataFrame(columns=['Investor Name', 'Lead Investor', 'Funding Round', 'Partners'])
                df_investors = df_investors.append({'Investor Name': 'none', 'Lead Investor': 'none', 'Funding Round': 'none', 'Partners': 'none'}, ignore_index=True)
            elif len(df_list)==2:
                df_comp = df_list[0]
                df_investors = df_list[1]
            elif len(df_list)>2:
                df_comp = df_list[0]
                df_investors = df_list[1]
                
        elif len(ok_clickable2)==2 : ## Two tables are not full length and need to be unwrapped
            
            ######## Start unwrap the list and charge the data#########################
            ok_clickable2[0].click()
            sleep(randint(2,3))
            page_html2 = driver.page_source
            df_list2 = pd.read_html(page_html2)
            df_comp = df_list2[0]
            ###### Stop unwrap the list and charge the data #########################
            driver.back()
            sleep(randint(2,3))
            page_html3 = driver.page_source
            
            ######## Start unwrap the list and charge the second data #########################
            clickables3 = driver.find_elements_by_class_name('mat-button-wrapper')
            ok_clickables3 =[]
            for clickable in clickables3:
                if clickable.text.strip()=='VIEW ALL':
                    ok_clickables3.append(clickable)
            ok_clickables3[1].click()
            sleep(randint(2,3))
            page_html4 = driver.page_source
            df_list4 = pd.read_html(page_html4)
            df_investors = df_list4[0]
            ######## Stop unwrap the list and charge the second data #########################
            driver.back()
            
        elif len(ok_clickable2)==1 : # only one of the tables need to be unwrapped
            ok_clickable2[0].click()
            sleep(randint(2,3))
            page_html = driver.page_source
            header = driver.find_elements_by_class_name('section-title')
            if (header[0].text.strip())=='Funding Rounds': ### Check which one it is that was unwrapped (investment vs investors)
                df_list = pd.read_html(page_html)
                df_comp = df_list[0]
                driver.back()
                sleep(randint(2,3))
                page_html = driver.page_source
                df_list = pd.read_html(page_html)
                df_investors = df_list[1]
            elif (header[0].text.strip())=='Investors':
                df_list = pd.read_html(page_html)
                df_investors = df_list[0]
                driver.back()
                sleep(randint(2,3))
                page_html = driver.page_source
                df_list = pd.read_html(page_html)
                df_comp = df_list[0]
                
        ##########################################################################################
        ################ Stop grabing data #######################################################
        ##########################################################################################
        
        ###########################################################################################
        #################### Compile and append into empty lists ##################################
        ###########################################################################################
        
        
    df_comp = formating_dates(df_comp) #### Formate dates
    df_comp = clean_up_dataset(df_comp) ### turn data from string and currency to integers
    
    
    comp_name = [companies[i]]*len(df_comp)
    comp_country = [country_agg[i]]*len(df_comp)
    comp_subsector = [Label[i]]*len(df_comp)
    comp_status = [status]*len(df_comp)
    df_comp['Comp_name'] = comp_name
    df_comp['Comp_country'] = comp_country
    df_comp['Comp_subsector'] = comp_subsector
    df_comp['Comp_status'] = comp_status
    list_df_comp.append(df_comp)
    list_df_investors.append(df_investors)
    sleep(randint(4,5))
    i = i+1

2
2
2


# Append the datasets that were stored in lists into a single dataset

In [21]:
# For investments
data_comp = pd.concat(list_df_comp)

In [1]:
# Uncomment to visualize
#data_comp

In [24]:
# For investors
df_investors = pd.concat(list_df_investors)

In [ ]:
# Separate comp name and investments
all_in = df_investors['Funding Round'].tolist()

funding_round = []
company = []
for item in all_in:
    if ' - ' in item:
        funding = item.split(' - ',1)[0]
        funding_round.append(funding)
        comp = item.split(' - ',1)[1]
        company.append(comp)
    else:
        funding_round.append('none')
        company.append('none')
df_investors['Round'] = funding_round
df_investors['Company_name'] = company

##########################################################################################
##### Add the country of the company the investors invested in by using data_comp#########
##########################################################################################

#### Create a subset of data_comp to work with by droping the duplicate hence  ############
#### the companies are unique and associated to their country #############################
df_companies_work = data_comp[['Comp_name', 'Comp_country']]
df_companies_work = df_companies_work.drop_duplicates()

####### Create two list that we will match from df_investor we take comp name, also from data_comp + the country from data comp#

companies_inv = df_investors['Company_name'].tolist()
comp_name = df_companies_work['Comp_name'].tolist()
comp_country = df_companies_work['Comp_country'].tolist()

Comp_country = []
for comp in companies_inv: # Takes each comp name in the investor dataset
    if comp in comp_name: # If the name is found in the data_comp dataset
        ind = (comp_name.index(comp)) # --> check the index
        country = comp_country[ind]# --> Find the country associated to this index into country name from data_comp
        Comp_country.append(country) # --> append it to list
    else:
        country = 'Unknown'
        Comp_country.append(country)
        
df_investors['Comp_country'] = Comp_country ## Add it to the dataset

In [26]:
data_invest.to_excel('C:/yourway/data_investors.xlsx', index=None, header= True)

In [3]:
#data_invest_made = pd.concat(list_df_investments_made)

In [4]:
#data_invest_made

In [5]:
#data_acquisition = pd.concat(list_acquisitions)
#data_acquisition

# Individuating the funding round from company name for investments data

In [ ]:
sep_name_round = data_comp['Transaction Name'].tolist()
funding_round = []
for item in sep_name_round:
    if ' - ' in item:
        funding = item.split(' - ',1)[0]
        funding_round.append(funding)
    else:
        funding_round.append('none')
data_comp['Round'] = funding_round
data_comp.drop(['Transaction Name'], axis=1)

# Checking which kind of funding there was and aggregate into highr classes to represent early risky investments - later stage investments - other kind of investments

In [ ]:
data_comp.Round.unique()

In [ ]:
Round = df_companies_clean.Round.tolist()
agg_round = []
for item in Round:
    if item =='Seed Round':
        agg_round.append('Seed')
    elif item == 'Series A':
        agg_round.append('Serie A')
    elif item == 'Funding Round':
        agg_round.append('Unknown type')
    elif item == 'Venture Round':
        agg_round.append('Serie A')
    elif item == 'Debt Financing':
        agg_round.append('Debt Financing')
    elif item == 'Pre Seed Round':
        agg_round.append('Seed')
    elif item == 'Grant':
        agg_round.append('Grant')
    elif item == 'Angel Round':
        agg_round.append('Angel')
    elif item == 'Non Equity Assistance':
        agg_round.append('Other')
    elif item == 'Convertible Note':
        agg_round.append('Seed')
    elif item == 'Series C':
        agg_round.append('C+')
    elif item == 'Series B':
        agg_round.append('Serie B')
    elif item == 'Corporate Round':
        agg_round.append('Serie B')
    elif item == 'Equity Crowdfunding':
        agg_round.append('Other')
    elif item == 'none':
        agg_round.append('none')
    elif item == 'Private Equity Round':
        agg_round.append('C+')
    elif item == 'Secondary Market':
        agg_round.append('IPO+')
    elif item == 'Series J':
        agg_round.append('C+')
    elif item == 'Series I':
        agg_round.append('C+')
    elif item == 'Series H':
        agg_round.append('C+')
    elif item == 'Series G':
        agg_round.append('C+')
    elif item == 'Series F':
        agg_round.append('C+')
    elif item == 'Series E':
        agg_round.append('C+')
    elif item == 'Series D':
        agg_round.append('C+')
    elif item == 'Post-IPO Equity':
        agg_round.append('IPO+')
    elif item == 'Post-IPO Debt':
        agg_round.append('IPO+')
    elif item == 'Post-IPO Secondary':
        agg_round.append('IPO+')
    elif item == 'Product Crowdfunding':
        agg_round.append('Other')
    elif item == 'Initial Coin Offering':
        agg_round.append('Other')
df_companies_clean['Agg_round'] = agg_round

In [23]:
#Save data
data_comp.to_excel('C:/yourway/data_invest.xlsx', index=None, header= True)

# Create two new datasets : One that takes off companies where no investments where made, one that discard missing values (if the amount of investment is not known)

In [ ]:
# Discard companies with no investments
df_companies_clean = data_comp[data_comp['Money_clean']!='0']

In [ ]:
# Discard missing values
df_comp_wmv = df_companies_clean[df_companies_clean['Money_clean']!='NaN']

# Convert float to integers
df_comp_wmv['Money_clean'] = df_comp_wmv['Money_clean'].astype(int)

In [ ]:
# save datasets
df_comp_wmv.to_excel('C:/your_path/data_private_investment_wmv.xlsx', index=False) ## Will be used to analyse the investments in quantity
df_companies_clean.to_excel('C:/your_path/data_private_investment_mv.xlsx', index=False) ## Will be used to analyse the investments in number